# Week XII - Trajectories: Dynamics Visualization and Radial Distribution Function

In this tutorial we analyze **trajectories** obtained by Monte Carlo (MC) simulations, in particular we focus on the visualization of the dynamics and the calculation of the radial distribution function (RDF). We will consider MC simulations of 2D hard disks, but the same considerations applies to molecular dynamics (MD) and other systems.

> We will analyse the trajectories with a post-processing approach:
> * The Fortran code `hd-MC.f90` will be use to run the dynamics. 
> * We have slightly modified `hd-MC.f90` to store the trajectory in a standardized format (ext XYZ)
> * We will use Python to interface with existing visualization libraries and calculate the RDF on top of the stored `.xyz` trajectories

NB: we will use the Python package `ase` for parsing and visualization (you can install it with by ``pip install ase``)

## Visualizaing the trajectory

* MC and MD simulations typically involve the calculation of a number of quantities for analysis, such as the RDF, the mean square displacement (MSD), the autocorrelation function and more.
* While these properties can be calculated "on the fly" during the simulations, it is often more convenient to store the trajectories and calculate the desired quantitites later. That mirrors exactly the situation in laboratory experiments: **data acquisition and data analysis are typically performed at different times**.
* The same data analysis routines can be used to study different systems, potentially obtained with different simulations codes, as long as a standardized *data format* is used.
* Hence, it is often very powerful to produce outputs in widely recognized formats and extensions, that allows to use existing packages for data analysis: you (or someone else) might have already produced the relevant function or subroutine.

### Extended XYZ and ASE
- We will use the extend XYZ format for storing the trajectories (see for instance https://www.ovito.org/docs/dev/reference/file_formats/input/xyz.html)
- We will use the Python package Atomic Simulation Environment (ASE) for reading the XYZ file (https://wiki.fysik.dtu.dk/ase/) and visualize the dynamics

### Simulations and post processing
1. First we look the code ``hd-MC_v2.f90``, which has been modified to store trajectories in the extended XYZ format (now we move to the code editor window)
2. Let's run in the terminal some MC with the Fortran code `hd-MC_v2.x` (N=30, 6x5, 500 steps for equilibration, 5000 for sampling)
3. Now we analyze the trajectoy ``traj.xyz``

In [3]:
import numpy as np
from ase.io import read
# We load and parse the trajectory
traj = read('./Fortran_code/traj.xyz',index=':')

ImportError: DLL load failed while importing _fblas: Impossibile trovare il modulo specificato.

In [ ]:
# ASE has converted the ext XYZ format in an internal format
# Each element of the list is a snapshot of the trajectory
traj

In [ ]:
a = traj[0]
print(a)

In [ ]:
_ = [a.set_pbc([True,True,False]) for a in traj]

In [ ]:
from ase.visualize import view
# Here we visualize the trajectory
view(traj)

In [ ]:
from ase.visualize.plot import plot_atoms
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
sn = traj[500]
plot_atoms(sn,ax,radii=[0.5]*sn.get_global_number_of_atoms())

In [ ]:
#sn = traj[501]
#plot_atoms(sn,ax,colors=['orange']*sn.get_global_number_of_atoms(),radii=[0.5]*sn.get_global_number_of_atoms())

## Radial Distribution Function (RDF)

* The RDF in 2D counts the number of particle pairs which are separated by a distance between $r$ and $r+dr$

$$g(r) = \frac{2}{2\pi r \Delta r \rho N}\langle \sum_{i=1}^{N-1}\sum_{j>i}\delta(r-r_{ij}) \rangle$$

> Now we implement the RDF calculation on top of the stored trajectories through a small Python function.

In [ ]:
def calculate_g(traj,dr = 0.05):
    '''
    Calculates the 2D radial distribution function (RDF)
    g(r) of a trajectory (list of ASE Atoms objects)
    '''
    Lx = traj[0].cell[0,0]
    Ly = traj[0].cell[1,1]
    #We create a list of Nx2 arrays
    traj_2D = [a.positions[:,:2] for a in traj]
    N = traj[0].get_global_number_of_atoms()
    print(Lx,Ly,N)
    def distance(dx, dy):
        dx = dx - Lx*int(2/Lx*dx)
        dy = dy - Ly*int(2/Ly*dy)
        return dx**2 + dy**2

    nmcs = len(traj_2D)
    ngcum = 0
    nbin = int(3.5/dr)
    print('{} bins'.format(nbin))
    gcum = np.zeros(nbin)

    for sn in traj_2D:
        x = sn[:,0] 
        y = sn[:,1]
        for i in range(N-1):
            for j in range(i+1,N):
                dx = x[i] - x[j]
                dy = y[i] - y[j]
                r2 =  distance(dx,dy)
                ibin = int(np.sqrt(r2)/dr) +1
                if (ibin<nbin):
                    gcum[ibin] = gcum[ibin] + 1 
    rho = float(N)/(Lx*Ly)
    xnorm =2.0/(rho*N*nmcs)
    rl,gl = [],[]    
    for ir in range(nbin):
        r    = ir*dr + 0.5*dr  
        area = 2.0*np.pi*r*dr    
        g    = gcum[ir]*xnorm/area
        rl.append(r)
        gl.append(g)
    return rl, gl


In [ ]:
rl,gl = calculate_g(traj)

In [ ]:
#Plotting
plt.plot(rl,gl,label='Post-processing RDF')
plt.xlabel('r')
plt.ylabel('g(r)')
#plt.plot(data[:,0],data[:,1])
plt.xlim([0,3.3])
plt.legend()
plt.axhline(1,color='grey',alpha=0.5)

> Let's comparare it with the one calculated during the run by the Fortran code.

In [ ]:
data = np.loadtxt('./Fortran_code/g_of_r.dat')
#Plotting
plt.plot(rl,gl,'-',label='Post-processing RDF')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.xlim([0,3.3])
plt.plot(data[:,0],data[:,1], 'o',alpha=0.5,label='On-the-fly RDF')
plt.legend()
plt.axhline(1,color='grey',alpha=0.5)

> Let's run some MC with the Fortran code `hd-MC_v2.x`for higher densities 
(N=16, 4.2x3.63, 500 steps for equilibration, 5000 for sampling)

In [ ]:
traj2 = read('./Fortran_code/traj2.xyz',index=':')
rl2,gl2 = calculate_g(traj2,dr=0.005)
#data = np.loadtxt('./Fortran_code/g_of_r.dat')
plt.plot(rl2,gl2,'-',label='Post-processing RDF')
#plt.plot(data[:,0],data[:,1], 'o',alpha=0.5,label='On-the-fly RDF')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.xlim([0,3.3])
plt.legend()
plt.axhline(1,color='grey',alpha=0.5)

In [ ]:
fig, ax = plt.subplots()
sn = traj2[500]
plot_atoms(sn,ax,radii=[0.5]*sn.get_global_number_of_atoms())

> Let's run some MC with the Fortran code `hd-MC_v2.x`for intermediate densities 
(N=16, 4.45x3.85, 500 steps for equilibration, 5000 for sampling)

In [ ]:
traj3 = read('./Fortran_code/traj3.xyz',index=':')
rl3,gl3 = calculate_g(traj3,dr=0.005)
plt.plot(rl,gl,'-',label=r'Post-processing RDF ($\rho=0.49$)')
plt.plot(rl2,gl2,'-',label=r'Post-processing RDF ($\rho=0.91$)')
plt.plot(rl3,gl3,'-',label=r'Post-processing RDF ($\rho=0.81$)')
plt.xlabel('r')
plt.ylabel('g(r)')
plt.xlim([0,2.5])
plt.legend()
plt.axhline(1,color='grey',alpha=0.5)

In [ ]:
fig, ax = plt.subplots()
sn = traj3[500]
plot_atoms(sn,ax,radii=[0.5]*sn.get_global_number_of_atoms())

In [ ]:
view(traj3)